In [1]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime

# Script summary
- Import several Excel files from D365 that lists sales orders with Load IDs and consolidate them in one dataframe
- Ask the user how many sessions this should be split into
- Split the dataframe into n x csv files
- Convert the CSV files to Excel

### Clean up the xml_prep folder

In [2]:
path = "data/rel_to_wh/outbound_to_EA/"
dir_list = os.listdir(path)

for i in range(0,len(dir_list)):
    xml_file = path+dir_list[i]
    os.remove(xml_file)

### Import files from D365 containing sales order lines with Load IDs

In [3]:
path = "data/rel_to_wh/inbound_from_D365/"
dir_list = os.listdir(path)

In [4]:
loadfiles_concat = pd.read_excel(path+dir_list[0])
try:
    loadfiles_concat.rename(columns={'Load ID': 'LoadID'}, inplace=True)
except:
    pass

In [5]:
for i in range(1,len(dir_list)):
    xml_file = path+dir_list[i]
    print(str(i)+xml_file)
    temp = pd.read_excel(path+dir_list[i])
    try:
        temp.rename(columns={'Load ID': 'LoadID'}, inplace=True)
    except:
        pass
    loadfiles_concat = pd.concat([loadfiles_concat, temp], ignore_index=True)

1data/rel_to_wh/inbound_from_D365/ZA1-000000811.xlsx
2data/rel_to_wh/inbound_from_D365/ZA1-000000799.xlsx
3data/rel_to_wh/inbound_from_D365/ZA1-000000806.xlsx
4data/rel_to_wh/inbound_from_D365/ZA1-000000812.xlsx
5data/rel_to_wh/inbound_from_D365/ZA1-000000805.xlsx
6data/rel_to_wh/inbound_from_D365/ZA1-000000809.xlsx
7data/rel_to_wh/inbound_from_D365/ZA1-000000807.xlsx
8data/rel_to_wh/inbound_from_D365/ZA1-000000802.xlsx
9data/rel_to_wh/inbound_from_D365/ZA1-000000804.xlsx
10data/rel_to_wh/inbound_from_D365/ZA1-000000801.xlsx
11data/rel_to_wh/inbound_from_D365/ZA1-000000803.xlsx
12data/rel_to_wh/inbound_from_D365/ZA1-000000810.xlsx
13data/rel_to_wh/inbound_from_D365/ZA1-000000808.xlsx


In [6]:
loads_to_wh = loadfiles_concat[['LoadID','Description']].copy()
loads_to_wh.drop(columns={'Description'}, inplace=True, axis=1)

In [7]:
loads_to_wh.drop_duplicates(keep='first',inplace=True)
loads_to_wh = loads_to_wh.dropna()
loads_to_wh.reset_index(drop=True, inplace=True)

In [8]:
loads_to_wh['LoadID'] = loads_to_wh['LoadID'].astype(int)
loads_to_wh['LoadID'] = loads_to_wh['LoadID'].astype(str)

In [9]:
loads_to_wh

,LoadID
0,2200011903
1,2200011989
2,2200011845
3,2200011846
4,2200011847
...,...
140,2200011956
141,2200011957
142,2200011988
143,2200011984


### Split the data among the number of user sessions

In [10]:
files_str = input('Enter the number of files into which the loads must be split')
files = int(files_str)

In [11]:
for file in range(1,files+1):
    with open('data/rel_to_wh/outbound_to_EA/wh' + str(file) + '.csv', 'a') as fw:        
        fw.write('LoadID'+'\n')

file = 0
for i in range(len(loads_to_wh)):
    if file <= files-1:
        file = file + 1
    else:
        file = 1
    with open('data/rel_to_wh/outbound_to_EA/wh' + str(file) + '.csv', 'a') as fw:        
        fw.write(str(loads_to_wh.loc[i, "LoadID"])+'\n')

### Now convert the CSV files into Excel

In [12]:
path = "data/rel_to_wh/outbound_to_EA/"
dir_list = os.listdir(path)

In [13]:
# 21 user - Release to warehouse , picking and despatch (Roadnet loads).xlsx

for i in range(0,len(dir_list)):
    excel_file = path+str(i+1)+' user - Release to warehouse , picking and despatch (Roadnet loads).xlsx'
    #print(str(i)+xml_file)
    temp = pd.read_csv(path+dir_list[i])
    temp.to_excel(excel_file, index=False)
    os.remove(path+dir_list[i])

In [15]:
#raise SystemExit("Stop right here!")